In [1]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import numpy as np

In [2]:
df = pd.read_csv('data_withID.csv')
df.head()

,PPSORT,AGEIMM,BedRm,CMA,CONDO,COW,DIST,GENSTAT,HHSIZE,IMMCAT5,...,Wages,WKSWRK,aboriginal,age,STEM,YrEd,PrMinWage_V,PrMinWage_Vstat,PartTime,Married
0,218995,9,1,835,1,4,7,1,2,21,...,2000,44.0,0,47,0,18,11.20,11.20,0,1
1,671467,0,3,933,0,1,7,3,5,1,...,28000,50.5,0,37,0,14,10.45,10.45,0,1
2,112607,0,2,462,0,1,22,2,3,1,...,36000,50.5,0,27,1,13,10.75,10.75,0,1
3,153693,9,2,535,0,1,12,1,4,21,...,80000,50.5,0,52,0,14,11.25,11.25,0,1
4,468158,0,2,535,0,1,2,2,4,1,...,11000,50.5,0,42,0,9,11.25,11.25,1,1


In [3]:
#Mincer earnings function
df["YrEd2"] = df["YrEd"]^2

In [4]:
df.columns

Index(['PPSORT', 'AGEIMM', 'BedRm', 'CMA', 'CONDO', 'COW', 'DIST', 'GENSTAT',
       'HHSIZE', 'IMMCAT5', 'MODE', 'MTNEn', 'MTNFr', 'NAICS', 'NOC16', 'NOCS',
       'PKIDS', 'POWST', 'PR', 'PRIHM', 'PWDUR', 'PWLEAVE', 'ROOMS', 'Sex',
       'SHELCO', 'Subsidy', 'Tenur', 'VisMin', 'Wages', 'WKSWRK', 'aboriginal',
       'age', 'STEM', 'YrEd', 'PrMinWage_V', 'PrMinWage_Vstat', 'PartTime',
       'Married', 'YrEd2'],
      dtype='object')

In [5]:
y = df["Wages"]
X = df[['age','YrEd','YrEd2','STEM','PrMinWage_V','PartTime','Sex','VisMin','Married']]

In [6]:
#get_dummies_CMA
dummies_CMA = pd.get_dummies(df.CMA)
CMA_dummies = dummies_CMA.set_axis(["Halifax","Moncton – Saint John","Québec","Montréal",
                                    "Sherbrooke – Trois-Rivières","Ottawa – Gatineau","Oshawa",
                                    "Toronto","Hamilton","St. Catharines – Niagara",
                                    "Kitchener – Cambridge – Waterloo","London","Windsor",
                                    "Brantford – Guelph – Barrie","Kingston – Peterborough",
                                    "Greater Sudbury / Grand Sudbury – Thunder Bay","Winnipeg",
                                    "Regina – Saskatoon","Calgary","Edmonton","Vancouver",
                                    "Victoria","Kelowna – Abbotsford","Non-CMA"], axis=1, inplace=False)
#X = X.join(CMA_dummies)

In [7]:
#get_dummies_NOC16
dummies_NOC16 = pd.get_dummies(df.NOC16)
#X = X.join(dummies_NOC16)

In [8]:
X.columns

Index(['age', 'YrEd', 'YrEd2', 'STEM', 'PrMinWage_V', 'PartTime', 'Sex',
       'VisMin', 'Married'],
      dtype='object')


Regression


In [9]:
model = LinearRegression()

In [10]:
model.fit(X,y)

LinearRegression()

In [11]:
y_pred = model.predict(X)
print(y_pred.shape)

(11256,)


In [12]:
print(model.coef_)

[ 4.73502484e+02  2.10767846e+03  2.15573026e+01  6.69407038e+03
  5.81970392e+03 -2.15802949e+04  1.02770327e+04 -8.70971748e+03
  3.38089591e+03]


In [13]:
model.intercept_

-73127.78012214354

In [14]:
variable = pd.DataFrame(X.columns)
coefficient = pd.DataFrame(model.coef_)

In [15]:
coefficient = pd.DataFrame(list(zip(X.columns, model.coef_)),
               columns =['variable', 'coefficient'])
intercept_df = pd.DataFrame({'variable': ["Intercept"],'coefficient': [model.intercept_]})
result = coefficient.append(intercept_df)

In [16]:
#result['coefficient'] = result['coefficient'].apply(lambda x:'%.08f' % x)
result

,variable,coefficient
0,age,473.502484
1,YrEd,2107.678459
2,YrEd2,21.557303
3,STEM,6694.070381
4,PrMinWage_V,5819.703917
5,PartTime,-21580.294910
6,Sex,10277.032704
7,VisMin,-8709.717485
8,Married,3380.895913
0,Intercept,-73127.780122


In [ ]:
q_age = int(input("enter your age: "))
q_YrEd = int(input("enter your total education years: "))
q_STEM = int(input("enter \'1\' if your field of study was considered to be STEM: "))
q_PrMInWage = float(input("enter the minimum wage in your location: "))
q_PartTime = int(input("enter \'1\' if you are part time worker: "))
q_Sex = int(input("enter \'1\' if you identify as male: "))
q_Married = int(input("enter \'1\' if you are currently in an marriage: "))
wage_predicted = (model.intercept_ + 
                  q_age*model.coef_[0] +
                  q_YrEd*model.coef_[1] +
                  (q_YrEd^2)*model.coef_[2] +
                  q_STEM*model.coef_[3] +
                  q_PrMInWage*model.coef_[4] +
                  q_PartTime*model.coef_[5] +
                  q_Sex*model.coef_[6] +
                  q_Married*model.coef_[7])
print("Your predicted yearly wage is: $" + str(wage_predicted))